In [22]:
import pandas as pd
import numpy as np
import glob, os, sys

osp = sys.platform
# import matplotlib.pyplot as plt
# %matplotlib inline

from utilities import PlotGame, LoadData, make_video
from helpers import id_player, id_position, id_teams

# ---------------------------------------------------------
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
gameid='0021500463'

# directories
# CHANGE HERE
main_dir = '../'
game_dir = main_dir+'data/'
# court_path = data_dir + 'nba_court_T.png'

Data = LoadData(main_dir, game_dir)
data = Data.load_game(gameid)

In [62]:
event_number = 145
# get events and number of events
event_df = pd.DataFrame(data['events'])
n_events = event_df.shape[0]
# get numebr of moments
moments = event_df.iloc[event_number,:]['moments']
n_moments = len(moments)

print(n_moments, event_df.shape)
event_df.head()

563 (231, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,668.42,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351444029, 692.25, 12.21, None, [[-1,...","[2, 3]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,692.25,"{'abbreviation': 'TOR', 'players': [{'playerid..."
3,665.71,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351463029, 673.22, 21.44, None, [[-1,...","[4, 5]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,673.22,"{'abbreviation': 'TOR', 'players': [{'playerid..."
4,646.64,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351473027, 668.0, 22.9, None, [[-1, -...",[6],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,668.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."


In [63]:
Plot = PlotGame(gameid, main_dir, game_dir)
# save all moments for a specific events
moments = range(n_moments)
for i in moments:
    print(i, end='\r')
    Plot.load_moment2img(data, event_number, i)

In [64]:
# prepare to save videos
event_dir = game_dir + 'game' + str(gameid) + '/' + 'event' + str(event_number) + '/'
video_name = event_dir + str(event_number) + '.mp4'
print(video_name)
images = glob.glob(event_dir + "*.png")
if osp == 'linux':
    # this is pretty hard-coded, should change
    images = sorted(images, key=lambda x: int(x.split('.')[2].split('/')[-1]))
else:
    images = sorted(images, key=lambda x: int(x.split('\\')[-1].split('.')[0]))
# print(images)
# save to video
make_video(images, video_name, 15)

../data/game0021500463/event145/145.mp4
The output video is ../data/game0021500463/event145/145.mp4


In [19]:
event_df.loc[0,:].playbyplay

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,TEAM,time_left,event_str
2,0021500463,2,1,1,1,8:11 PM,11:46,None,None,DeRozan 15' Jump Shot (2 PTS),...,0,0,None,NaN,None,None,None,TOR,706,make


In [54]:
event_df.playbyplay[0].to_dict(orient='records')

[{'EVENTMSGACTIONTYPE': 1,
  'EVENTMSGTYPE': 1,
  'EVENTNUM': 2,
  'GAME_ID': '0021500463',
  'HOMEDESCRIPTION': None,
  'NEUTRALDESCRIPTION': None,
  'PCTIMESTRING': '11:46',
  'PERIOD': 1,
  'PERSON1TYPE': 5.0,
  'PERSON2TYPE': 0,
  'PERSON3TYPE': 0,
  'PLAYER1_ID': 201942,
  'PLAYER1_NAME': 'DeMar DeRozan',
  'PLAYER1_TEAM_CITY': 'Toronto',
  'PLAYER1_TEAM_ID': 1610612761.0,
  'PLAYER1_TEAM_NICKNAME': 'Raptors',
  'PLAYER2_ID': 0,
  'PLAYER2_NAME': None,
  'PLAYER2_TEAM_ABBREVIATION': None,
  'PLAYER2_TEAM_CITY': None,
  'PLAYER2_TEAM_ID': nan,
  'PLAYER2_TEAM_NICKNAME': None,
  'PLAYER3_ID': 0,
  'PLAYER3_NAME': None,
  'PLAYER3_TEAM_ABBREVIATION': None,
  'PLAYER3_TEAM_CITY': None,
  'PLAYER3_TEAM_ID': nan,
  'PLAYER3_TEAM_NICKNAME': None,
  'SCORE': '2 - 0',
  'SCOREMARGIN': '-2',
  'TEAM': 'TOR',
  'VISITORDESCRIPTION': "DeRozan 15' Jump Shot (2 PTS)",
  'WCTIMESTRING': '8:11 PM',
  'event_str': 'make',
  'time_left': 706}]

In [21]:
event_df.loc[0, :].moments

[[1,
  1451351428029,
  708.28,
  12.78,
  None,
  [[-1, -1, 18.38063, 14.07976, 8.56325],
   [1610612741, 2200, 11.15334, 21.35529, 0.0],
   [1610612741, 201959, 16.79035, 20.55978, 0.0],
   [1610612741, 201565, 9.12233, 39.32051, 0.0],
   [1610612741, 202710, 21.15543, 32.71616, 0.0],
   [1610612741, 202703, 8.40459, 11.67492, 0.0],
   [1610612761, 2449, 24.18381, 44.21187, 0.0],
   [1610612761, 201960, 1.06327, 2.54971, 0.0],
   [1610612761, 200768, 7.28146, 48.40417, 0.0],
   [1610612761, 201942, 18.1243, 14.25539, 0.0],
   [1610612761, 202687, 10.82794, 26.70275, 0.0]]],
 [1,
  1451351428069,
  708.24,
  12.75,
  None,
  [[-1, -1, 18.29249, 14.14733, 8.97331],
   [1610612741, 2200, 10.99227, 21.31452, 0.0],
   [1610612741, 201959, 16.47709, 20.40799, 0.0],
   [1610612741, 201565, 9.04023, 39.29879, 0.0],
   [1610612741, 202710, 21.28172, 32.89095, 0.0],
   [1610612741, 202703, 8.39265, 11.69472, 0.0],
   [1610612761, 2449, 24.25942, 44.12444, 0.0],
   [1610612761, 201960, 1.1199, 

In [27]:
# event_df

In [26]:
id_team = id_teams([event_df])
id_team

{1610612741: 'chicago bulls', 1610612761: 'toronto raptors'}

In [50]:
hid = 1610612741
vid = 1610612761
hteam_state = []
vteam_state = []
for i in event_df.loc[0, :].moments:
    ai = np.array(i[5])
    assert len(ai) == 11
    bi = np.array(i[5][0][2:4])
    ha = ai[ai[:, 0]==hid, 2:4]
    va = ai[ai[:, 0]==vid, 2:4]
#     print(bi, ha, va)
    hd = (np.sum((ha-bi)**2, axis=1)**0.5).min()
    vd = (np.sum((va-bi)**2, axis=1)**0.5).min()
    if hd > vd:
        hteam_state.append('defend')
        vteam_state.append('offend')
    else:
        hteam_state.append('offfend')
        vteam_state.append('defend')
#     break

In [68]:
# hteam_state

In [40]:
a = np.array([ [1,2], [0,3] ])
b = np.array([2, 3])
a

array([[1, 2],
       [0, 3]])

In [41]:
b

array([2, 3])

In [42]:
a - b

array([[-1, -1],
       [-2,  0]])

In [45]:
(a-b)**2

array([[1, 1],
       [4, 0]])

In [47]:
np.sum((a-b)**2, axis=1)**0.5

array([1.41421356, 2.        ])